In [ ]:
# Copyright 2021 NVIDIA CORPORATION
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
import glob
import os
import pandas as pd
import numpy as np
import gc
import datetime
import hashlib
import emoji
import re
import pickle
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
from transformers import BertTokenizer

In [2]:
all_features = [
    'text_tokens',    ###############
    'hashtags',       #Tweet Features
    'tweet_id',       #
    'media',          #
    'links',          #
    'domains',        #
    'tweet_type',     #
    'language',       #
    'timestamp',      ###############
    'a_user_id',              ###########################
    'a_follower_count',       #Engaged With User Features
    'a_following_count',      #
    'a_is_verified',          #
    'a_account_creation',     ###########################
    'b_user_id',              #######################
    'b_follower_count',       #Engaging User Features
    'b_following_count',      #
    'b_is_verified',          #
    'b_account_creation',     #######################
    'b_follows_a',    #################### Engagement Features
    'reply',          #Target Reply
    'retweet',        #Target Retweet    
    'retweet_comment',#Target Retweet with comment
    'like',           #Target Like
                      ####################
]

In [3]:
MAP_MEDIA = {
 '': 0,
 'Photo': 1,
 'Photo\tPho': 2,
 'Video': 3,
 'GIF': 4,
 'Video\tVid': 5,
 'Photo\tVid': 6,
 'Video\tPho': 7,
 'GIF\tPhoto': 8,
 'Photo\tGIF': 9,
 'GIF\tGIF': 10,
 'GIF\tVideo': 11,
 'Video\tGIF': 12,
 'GIF\tGIF\tG': 13
}

MAP_TYPE = {'TopLevel': 0, 'Retweet': 1, 'Quote': 2}

MAP_LANG = {
 '488B32D24BD4BB44172EB981C1BCA6FA': 0,
 'E7F038DE3EAD397AEC9193686C911677': 1,
 'B0FA488F2911701DD8EC5B1EA5E322D8': 2,
 'B8B04128918BBF54E2E178BFF1ABA833': 3,
 '313ECD3A1E5BB07406E4249475C2D6D6': 4,
 '1F73BB863A39DB62B4A55B7E558DB1E8': 5,
 '9FCF19233EAD65EA6E32C2E6DC03A444': 6,
 '9A78FC330083E72BE0DD1EA92656F3B5': 7,
 '8729EBF694C3DAF61208A209C2A542C8': 8,
 'E6936751CBF4F921F7DE1AEF33A16ED0': 9,
 '7F4FAB1EB12CD95EDCD9DB2A6634EFCE': 10,
 'B4DC2F82961F1263E90DF7A942CCE0B2': 11,
 '310ECD7D1E42216E3C1B31EFDDFC72A7': 12,
 '5A0759FB938B1D9B1E08B7A3A14F1042': 13,
 '2F548E5BE0D7F678E72DDE31DFBEF8E7': 14,
 '5B6973BEB05212E396F3F2DC6A31B71C': 15,
 '2573A3CF633EBE6932A1E1010D5CD213': 16,
 'DA13A5C3763C212D9D68FC69102DE5E5': 17,
 '00304D7356D6C64481190D708D8F739C': 18,
 '7D11A7AA105DAB4D6799AF863369DB9C': 19,
 '23686A079CA538645BF6118A1EF51C8B': 20,
 'A5CFB818D79497B482B7225887DBD3AD': 21,
 '838A92D9F7EB57FB4A8B0C953A80C7EB': 22,
 '99CA116BF6AA65D70F3C78BEBADC51F0': 23,
 'D922D8FEA3EFAD3200455120B75BCEB8': 24,
 '159541FA269CA8A9CDB93658CAEC4CA2': 25,
 'E84BE2C963852FB065EE827F41A0A304': 26,
 '6B90065EA806B8523C0A6E56D7A961B2': 27,
 '4B55C45CD308068E4D0913DEF1043AD6': 28,
 'BAC6A3C2E18C26A77C99B41ECE1C738D': 29,
 '4CA37504EF8BA4352B03DCBA50E98A45': 30,
 '3228B1FB4BC92E81EF2FE35BDA86C540': 31,
 'D7C16BC3C9A5A633D6A3043A567C95A6': 32,
 '477ED2ED930405BF1DBF13F9BF973434': 33,
 '41776FB50B812A6775C2F8DEC92A9779': 34,
 'C1E99BF67DDA2227007DE8038FE32470': 35,
 'F70598172AC4514B1E6818EA361AD580': 36,
 '6744F8519308FD72D8C47BD45186303C': 37,
 '10C6C994C2AD434F9D49D4BE9CFBC613': 38,
 '89CE0912454AFE0A1B959569C37A5B8F': 39,
 '105008E45831ADE8AF1DB888319F422A': 40,
 'DE8A3755FCEDC549A408D7B1EB1A2C9F': 41,
 'BF04E736C599E9DE22F39F1DC157E1F1': 42,
 'CF304ED3CFC1ADD26720B97B39900FFD': 43,
 '59BE899EB83AAA19878738040F6828F0': 44,
 '3DF931B225B690508A63FD24133FA0E2': 45,
 '3AB05D6A4045A6C37D3E4566CFDFFE26': 46,
 '678E280656F6A0C0C23D5DFD46B85C14': 47,
 '440116720BC3A7957E216A77EE5C18CF': 48,
 'A3E4360031A7E05E9279F4D504EE18DD': 49,
 'C41F6D723AB5D14716D856DF9C000DED': 50,
 '7E18F69967284BB0601E88A114B8F7A9': 51,
 'F9D8F1DB5A398E1225A2C42E34A51DF6': 52,
 '914074E75CB398B5A2D81E1A51818CAA': 53,
 '5B210378BE9FFA3C90818C43B29B466B': 54,
 'F33767F7D7080003F403FDAB34FEB755': 55,
 'DC5C9FB3F0B3B740BAEE4F6049C2C7F1': 56,
 '3EA57373381A56822CBBC736169D0145': 57,
 '37342508F52BF4B62CCE3BA25460F9EB': 58,
 '7168CE9B777B76E4069A538DC5F28B6F': 59,
 '0BB2C843174730BA7D958C98B763A797': 60,
 'CDE47D81F953D800F760F1DE8AA754BA': 61,
 '9D831A0F3603A54732CCBDBF291D17B7': 62,
 '5F152815982885A996841493F2757D91': 63,
 '82C9890E4A7FC1F8730A3443C761143E': 64,
 '8C64085F46CD49FA5C80E72A35845185': 65}

In [4]:
!rm -r /raid/recsys2021_token
!mkdir /raid/recsys2021_token

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
rm: cannot remove '/raid/recsys2021_token': No such file or directory
/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [5]:
files = glob.glob('/raid/recsys2021/*')
files = [file for file in files if '.lzo' not in file and 'training_urls.txt' not in file]
len(files)

253

In [6]:
def extract_feature(fn):

    df = pd.read_csv(fn, sep='\x01', header=None)
    df.columns = all_features

    filenumber = int(fn.split('/')[-1].split('-')[-1])

    #Only run in trainset and not in test
    if 'like' in df.columns: # do this file contains the target?
        df['reply'] = df['reply'].fillna(0)
        df['retweet'] = df['retweet'].fillna(0)
        df['retweet_comment'] = df['retweet_comment'].fillna(0)
        df['like'] = df['like'].fillna(0)    
        df.loc[df.reply>0,'reply'] = 1
        df.loc[df.retweet>0,'retweet'] = 1
        df.loc[df.retweet_comment>0,'retweet_comment'] = 1
        df.loc[df.like>0,'like'] = 1
        df['reply'] = df['reply'].astype(np.int8)
        df['retweet'] = df['retweet'].astype(np.int8)
        df['retweet_comment'] = df['retweet_comment'].astype(np.int8)
        df['like'] = df['like'].astype(np.int8)
    
    df['media'] = df['media'].apply(lambda x: MAP_MEDIA[x[:9]] if x==x else 0).astype(np.int8)
    df['tweet_type'] = df['tweet_type'].apply(lambda x: MAP_TYPE[x] if x==x else 0).astype(np.int8)
    df['language'] = df['language'].apply(lambda x: MAP_LANG[x] if x==x else 0).astype(np.int8)
    
    df['timestamp'] = df['timestamp'].astype(np.uint32)
    
    df['tweet_id'] = df['tweet_id'].apply(lambda x: int(x[-16:],16) ).astype(np.int64)
    df['a_user_id'] = df['a_user_id'].apply(lambda x: int(x[-16:],16) ).astype(np.int64)
    df['b_user_id'] = df['b_user_id'].apply(lambda x: int(x[-16:],16) ).astype(np.int64)
    ##########################################################################################    
        
    df[[
        'text_tokens',
        'tweet_id',
        'a_user_id',
        'b_user_id',
        'timestamp',
        'media',
        'tweet_type',
        'language',
        'reply',          #Target Reply
        'retweet',        #Target Retweet    
        'retweet_comment',#Target Retweet with comment
        'like'
    ]].to_parquet( '/raid/recsys2021_token/' + fn.split('/')[-1] + '.parquet'  )

    return
    




In [7]:
for file in sorted(files):
    print(file)
    extract_feature(file)

/raid/recsys2021/part-00000
/raid/recsys2021/part-00001
/raid/recsys2021/part-00002
/raid/recsys2021/part-00003
/raid/recsys2021/part-00004
/raid/recsys2021/part-00005
/raid/recsys2021/part-00006
/raid/recsys2021/part-00007
/raid/recsys2021/part-00008
/raid/recsys2021/part-00009
/raid/recsys2021/part-00010
/raid/recsys2021/part-00011
/raid/recsys2021/part-00012
/raid/recsys2021/part-00013
/raid/recsys2021/part-00014
/raid/recsys2021/part-00015
/raid/recsys2021/part-00016
/raid/recsys2021/part-00017
/raid/recsys2021/part-00018
/raid/recsys2021/part-00019
/raid/recsys2021/part-00020
/raid/recsys2021/part-00021
/raid/recsys2021/part-00022
/raid/recsys2021/part-00023
/raid/recsys2021/part-00024
/raid/recsys2021/part-00025
/raid/recsys2021/part-00026
/raid/recsys2021/part-00027
/raid/recsys2021/part-00028
/raid/recsys2021/part-00029
/raid/recsys2021/part-00030
/raid/recsys2021/part-00031
/raid/recsys2021/part-00032
/raid/recsys2021/part-00033
/raid/recsys2021/part-00034
/raid/recsys2021/par

In [ ]:
res = Parallel(n_jobs=10,backend='multiprocessing')(delayed(extract_feature)(fn) for fn in tqdm(sorted(files)))
gc.collect()

  0%|          | 0/253 [00:00<?, ?it/s]